## Exemplo de classificador com leitura do CSV. 

Etapas:

- Leitura CSV.
- Tratamento de imagens com lib PIL. 
- Leitura imagem. 
- Aplicacao Filtro para remocao background


Input do modelo : (Imagem, sexo, idade, etnia)

In [ ]:
!pip install -r requirements.txt

In [1]:
import pandas as pd
from PIL import Image, ImageChops
import numpy as np
import torch
from lib.ImageFIlter import treat_image_PIL

In [20]:
ds_file = pd.read_csv('dataset/exemplo_csv_2.csv')


## Geracao Tensor com imagens filtradas

In [25]:
## Loop
img_dataset = np.ones((ds_file.shape[0],256,256),dtype=np.uint8)

j=0
for i in ds_file['path']:
    img_dataset[j]=treat_image_PIL('dataset/path/'+i,1)
    j+=1
tensor_imagem = torch.tensor(img_dataset)


## Geracao Tensor Age, sex, Etinia

Encoding: 

- Sexo: M=0 F=1

- Idade: Normalizar? Sim, dividir por 100 igual o artigo do Marco e Felipe fizeram. 

- Etnia: Encoding simples de categorias. Utilizaremos o Label Encoding para isso 

In [11]:
ds_id_sex_et= ds_file[['IDADE', 'SEXO', 'ETNIA']]
ds_id_sex_et['SEXO'] = ds_file['SEXO'].map({ 'M':0, 'F':1})
ds_id_sex_et['IDADE'] = ds_file['IDADE']/100

/var/folders/30/pgp77_r92w5_rn1ghx4kn51h0000gn/T/ipykernel_56713/1074131203.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_id_sex_et['SEXO'] = ds_file['SEXO'].map({ 'M':0, 'F':1})
/var/folders/30/pgp77_r92w5_rn1ghx4kn51h0000gn/T/ipykernel_56713/1074131203.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_id_sex_et['IDADE'] = ds_file['IDADE']/100


In [23]:
## Encoding variavel ETNIA -> Usando label Encoding
from sklearn.preprocessing import LabelEncoder

# Create a sample dataframe with categorical data

#print(f"Before Encoding the Data:\n\n{ds_id_sex_et['ETNIA']}\n")

# Create a LabelEncoder object
le = LabelEncoder()

# Fit and transform the categorical data
ds_id_sex_et['ETNIA'] = le.fit_transform(ds_id_sex_et['ETNIA'])
ds_id_sex_et

,IDADE,SEXO,ETNIA
0,0.85,0,1
1,0.32,1,1
2,0.60,1,1
3,0.57,1,4
4,0.57,1,1
...,...,...,...
211,0.35,1,3
212,0.72,0,2
213,0.71,1,2
214,0.49,0,2


## Geracao de rótulos para classificação

Colunas utilizadas para classificacao (Labels)

labels = [ 'FA', 'TA/Flutter',
       'TPSV/TS', 'TV/FV', 'BAV 2º/3º/Avanc./BS', 'Supra ST',
       'Corrente de Lesao', 'Extrassistole', 'BRD', 'BRE', 'MP', 'Normal',
       'Outros', 'Exclusão']

In [22]:
##
ds_file.columns
labels = [ 'FA', 'TA/Flutter',
       'TPSV/TS', 'TV/FV', 'BAV 2º/3º/Avanc./BS', 'Supra ST',
       'Corrente de Lesao', 'Extrassistole', 'BRD', 'BRE', 'MP', 'Normal',
    'Outros']
ds_labels = ds_file[labels]
ds_labels=ds_labels.astype(int)#,'False':0})
tensor_label = torch.tensor(np.array(ds_labels))
tensor_label

tensor([[0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        ...,
        [0, 1, 1,  ..., 0, 1, 0],
        [0, 1, 0,  ..., 0, 0, 1],
        [1, 0, 0,  ..., 1, 0, 0]])

In [24]:
ds_file['Normal']

0      False
1      False
2      False
3      False
4      False
       ...  
211    False
212     True
213     True
214    False
215    False
Name: Normal, Length: 216, dtype: bool

### Resumo:

model(tensor_imagem,ds_id_sex_et)

labels : tensor_label